In [1]:
from zhinst import ziPython
from numpy import *
from matplotlib import pylab as pl

from scipy import io as spio
from scipy import signal

from scipy import fftpack, ndimage
from ipywidgets import widgets


daq = ziPython.ziDAQServer('localhost',8004 )
device = 'dev2201'
daq.connectDevice('DEV2201','usb') 

daq.subscribe('/dev2201/demods/0/sample')
daq.subscribe('/dev2201/demods/1/sample')

In [13]:
# This block of code sets lockin-parameters. Make sure to play with these settings
# if you are using a new laser, or sweeping at a different rate


# # These seem optimum for the Agilent Laser, at 500 pm/s
# freq = 25e6       #Demodulation frequency
# datarate = 200000#Data Collection Rate
# timeconstant = 1/1500e3 #lowpass time constant (limits the scan speed)
# harmonics = [1,4] #Slects which harmonics of the demodulation freq to use (25MHz and 100Mhz in this case) 


# These seem optimum for the CTL Laser, at 1500 pm/s
freq = 25e6        #Demodulation frequency
datarate = 200000  #Data Collection Rate
timeconstant = 1/9000e3 #lowpass time constant (limits the scan speed)
harmonics = [1,4] #Slects which harmonics of the demodulation freq to use (25MHz and 100Mhz in this case) 



print('Demod channels at '+ ' ,'.join(['{:.0f} MHz'.format(i*freq/1e6) for i in harmonics]))

#configuration
## Set the measurement
for i in range(2):
    code = '/{:}/DEMODS/{:}/'.format(device,i)
    print(code)
    daq.set([(code+'rate',datarate),
             (code+'harmonic',harmonics[i]),
             (code+'order',2),
             (code+'timeconstant',timeconstant),
             (code+'enable',True),
     ])
    daq.subscribe(code+'sample')

Demod channels at 25 MHz ,100 MHz
/dev2201/DEMODS/0/
/dev2201/DEMODS/1/


## Start the measurement

In [64]:
#this collects data, namely R^2 of each lockin channel, and Aux in, which is the transmission

# take XX seconds of data, and saves it to filename.mat
scan_duration=50
filename='12_8_2016_Pratham_1530_1570_TE_type2a4r1_6'

# Will start collecting immediately after running this block

daq.flush()
ch1 = []
ch1_t = []
ch2 = []
ch2_t = []
trans = []
trans_t = []
for i in range(scan_duration): # collect X seconds of data
    data = daq.poll(1,1)
    ch1.append(data['dev2201']['demods']['0']['sample']['x']**2 + data['dev2201']['demods']['0']['sample']['y']**2)
    ch2.append(data['dev2201']['demods']['1']['sample']['x']**2 + data['dev2201']['demods']['1']['sample']['y']**2)
    trans.append(data['dev2201']['demods']['0']['sample']['auxin0'])
    ch1_t.append(data['dev2201']['demods']['0']['sample']['timestamp'])
    ch2_t.append(data['dev2201']['demods']['1']['sample']['timestamp'])
    trans_t.append(data['dev2201']['demods']['0']['sample']['timestamp'])
    
ch1 = hstack(ch1)
ch2 = hstack(ch2)
ch1_t = hstack(ch1_t)
ch2_t = hstack(ch2_t)
trans = hstack(trans)
trans_t = hstack(trans_t)


spio.savemat(filename+'.mat', {'ch1':vstack([ch1_t, ch1]),
                           'ch2':vstack([ch2_t, ch2]),
                           'trans':vstack([trans_t, trans]),
                      'freq': freq, 'channels': ['ch1','ch2'], 'harmonics': harmonics},
             do_compression = True)